# Car Tracker 


This notebook shows how to build a simple car dwell time tracker a pretrained SSD Mobilenet Tensorflow Object Detection Model and a Centroid Tracker.

By completing this notebook, you will learn:
* How to write a Python script for your app that takes in camera streams, performs inference, and outputs results
* How to use a Tensorflow Object Detection model with Panorama
* How to bundle additional Python files and libraries with your container
* How to build a simple car tracker

--- 

1. [Prerequisites](#Prerequisites)
1. [Set up](#Set-up)
1. [Download model](#Download-model)
1. [Build application](#Build-application)
1. [Upload application](#Upload-application)
1. [Deploy application](#Deploy-application)

# Prerequisites

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources on your behalf.

# Set Up
Import libraries for use with this notebook environment, you do not need these libraries when you write your application code. Run these 3 cells every time you update your app code and restart your kernel.

In [ ]:
import sys
import os
import time
import json

import boto3
import sagemaker

import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama')

# configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)

# register custom magic command
@register_cell_magic
def save_cell(line, cell):
    'Save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)

## Notebook parameters

Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
ML_MODEL_FNAME = 'ssd_mobilenet_v2_coco_tf_trt'

In [ ]:
app_name = 'car_tracker_app'
code_package_name = 'CAR_TRACKER_CODE'

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./car_tracker_app && panorama-cli import-application

# Download model

While working with the Panorama sample code, we provide pretrained models for you to use. Locally, our models need to be accessible by the application container, so we will store them in the `car_tracker/car_tracker_app/packages/(account_id)-CAR_TRACKER_CODE-1.0/src` folder. This step downloads the model artifacts from our Amazon S3 bucket to the local folder. If you want to use your own models, put your tar.gz file into the folder.

In [ ]:
# Downloads pretrained model for this sample.
# This step takes some time, depending on your network environment.
panorama_test_utility.download_sample_model( ML_MODEL_FNAME, f"./car_tracker_app/packages/{account_id}-{code_package_name}-1.0/src" )

# Unpack the downloaded model tarball
!cd ./car_tracker_app/packages/{account_id}-{code_package_name}-1.0/src && tar -xvzf {ML_MODEL_FNAME}.tar.gz && rm {ML_MODEL_FNAME}.tar.gz

### Build application

In [ ]:
container_asset_name = 'code_asset'

In [ ]:
%%capture captured_output

# Building container image.This process takes time (5min ~ 10min)
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./car_tracker_app && panorama-cli build \
    --container-asset-name {container_asset_name} \
    --package-path packages/{account_id}-{code_package_name}-1.0

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

### Special flags in package.json

* Step 1 : Before you deploy the application, open car_tracker/car_tracker_app/packages/(account-id)-CAR_TRACKER_CODE-1.0/package.json
* Step 2 : Add the following flags to the package.json

```
"requirements": 
            [{
                    "type" : "hardware_access",
                    "nvAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
            }]
```

The assets should look something like this

```
"assets": [
    {
        "name": "code_asset",
        "implementations": [
            {
                "type": "container",
                "assetUri": "9a49a98784f4571adacc417f00942dac7ef2e34686eef21dca9fcb7f4b7ffd70.tar.gz",
                "descriptorUri": "4bab130ec48eea84e072d9fe813b947e9d9610b2924099036b0165026a91d306.json",
                "requirements": 
                [{
                    "type" : "hardware_access",
                    "nvAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
                }]
            }
        ]
    }
],
```

### Upload application

In [ ]:
# This step takes some time, depending on your network environment.
!cd ./car_tracker_app && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, go to the Panorama console and deploy using the GUI. The GUI makes it easy to select camera streams and the devices that you want to deploy to.

#### Deploy Application

* Step 1: Copy the contents of `car_tracker/car_tracker_app/graphs/car_tracker_app/graph.json`
* Step 2 : Go to the AWS Panorama Console, click Deploy Application
* Step 3 : Paste the contents you just copied
* Step 4 : Select the target device to deploy to
* Step 5: Select the camera that you want to use
* Step 6 : Deploy!

This can take some time (~30 minutes), depending on your network conditions.